In [1]:
import mxnet as mx
from mxnet import nd
from mxnet import autograd
from mxnet import gluon
import numpy as np

In [2]:
ctx = mx.cpu()

In [3]:
import json

with open("./data/train_data_50000.json") as json_data:
    train_data = json.load(fp=json_data)
    
with open("./data/test_data_50000.json") as json_data:
    test_data = json.load(fp=json_data)

train_X = np.array(train_data['data'])
train_Y = np.array(train_data['label'])

test_X = np.array(test_data['data'])
test_Y = np.array(test_data['label'])

In [4]:
batch_size = 200
num_inputs = 64
num_outputs = 2
train_data = mx.io.NDArrayIter(train_X,train_Y ,
                               batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(test_X, test_Y,
                               batch_size, shuffle=True)

In [5]:
num_hidden = 256
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [6]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [7]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [8]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})


In [9]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [10]:
epochs = 40
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
            cross_entropy.backward()
        trainer.step(data.shape[0])
        if i == 0:
            moving_loss = nd.mean(cross_entropy).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Val_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.679267438143, Train_acc 0.6005, Val_acc 0.53262
Epoch 1. Loss: 0.666664345602, Train_acc 0.60732, Val_acc 0.53188
Epoch 2. Loss: 0.662091931207, Train_acc 0.6122, Val_acc 0.53248
Epoch 3. Loss: 0.658314183439, Train_acc 0.61658, Val_acc 0.53356
Epoch 4. Loss: 0.654410905468, Train_acc 0.62134, Val_acc 0.53532
Epoch 5. Loss: 0.650156286175, Train_acc 0.62686, Val_acc 0.53798
Epoch 6. Loss: 0.645669647083, Train_acc 0.63328, Val_acc 0.54122
Epoch 7. Loss: 0.641047437192, Train_acc 0.63826, Val_acc 0.54448
Epoch 8. Loss: 0.636384127089, Train_acc 0.6437, Val_acc 0.54886
Epoch 9. Loss: 0.631705665613, Train_acc 0.64954, Val_acc 0.55512
Epoch 10. Loss: 0.627166296163, Train_acc 0.65538, Val_acc 0.56102
Epoch 11. Loss: 0.62278748654, Train_acc 0.6603, Val_acc 0.56448
Epoch 12. Loss: 0.61852692226, Train_acc 0.66576, Val_acc 0.56918
Epoch 13. Loss: 0.614427657235, Train_acc 0.67096, Val_acc 0.57396
Epoch 14. Loss: 0.610358259661, Train_acc 0.67616, Val_acc 0.57676
Epoch 15. L